In [65]:
import pandas as pd
import datetime

In [66]:
# Source: http://web.mta.info/developers/turnstile.html
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    sample_counter=0
    for sample_num,week_num in enumerate(week_nums):
        file_url = url.format(week_num)
        temp_df=pd.read_csv(file_url)
        temp_df["sample_num"]=sample_num
        dfs.append(temp_df)
    return pd.concat(dfs)

In [67]:
week_nums = [200201, 200307, 200404,200502,200606,200704,200801\
             ,200905,201003,201107,201205,210102,210206,210306,210403]
turnstiles_df = get_data(week_nums)

In [68]:
turnstiles_df.head(23)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,sample_num
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/25/2020,03:00:00,REGULAR,7356095,2493703,0
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/25/2020,07:00:00,REGULAR,7356105,2493714,0
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/25/2020,11:00:00,REGULAR,7356170,2493761,0
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/25/2020,15:00:00,REGULAR,7356333,2493812,0
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/25/2020,19:00:00,REGULAR,7356581,2493862,0
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/25/2020,23:00:00,REGULAR,7356730,2493891,0
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/26/2020,03:00:00,REGULAR,7356770,2493906,0
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/26/2020,07:00:00,REGULAR,7356776,2493910,0
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/26/2020,11:00:00,REGULAR,7356826,2493953,0
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/26/2020,15:00:00,REGULAR,7356974,2493994,0


### cleaning


In [69]:
turnstiles_df.columns = [column.strip() for column in turnstiles_df.columns]
turnstiles_df["DATE_TIME"] = pd.to_datetime(turnstiles_df.DATE + " " + turnstiles_df.TIME, 
                                            format="%m/%d/%Y %H:%M:%S")
#consolodating coloumns                        
#turnstiles_df['unique_station']=turnstiles_df.['STATION']+"_"+turnstiles_df.LINENAME
#turnstiles_df['unique_turnstile']=turnstiles_df['UNIT']+turnstiles_df['C/A']+turnstiles_df['SCP']+turnstiles_df['unique_station']
#turnstiles_df['unique_turnstile'].value_counts().sort_index().count()

In [70]:
# Get rid of the duplicate entry
turnstiles_df.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], 
                          inplace=True, ascending=False)
turnstiles_df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)

In [71]:
turnstiles_df = turnstiles_df.drop(["EXITS", "DESC"], axis=1, errors="ignore")

### Calculating daily entries

In [72]:
turnstiles_daily = (turnstiles_df
                        .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE","sample_num"],as_index=False)
                        .ENTRIES.first())

In [73]:
turnstiles_daily.head(11)

,C/A,UNIT,SCP,STATION,DATE,sample_num,ENTRIES
0,A002,R051,02-00-00,59 ST,01/01/2021,11,7511647
1,A002,R051,02-00-00,59 ST,01/25/2020,0,7356730
2,A002,R051,02-00-00,59 ST,01/26/2020,0,7357340
3,A002,R051,02-00-00,59 ST,01/27/2020,0,7358737
4,A002,R051,02-00-00,59 ST,01/28/2020,0,7360060
5,A002,R051,02-00-00,59 ST,01/29/2020,0,7361355
6,A002,R051,02-00-00,59 ST,01/30/2020,0,7362672
7,A002,R051,02-00-00,59 ST,01/30/2021,12,7524821
8,A002,R051,02-00-00,59 ST,01/31/2020,0,7364037
9,A002,R051,02-00-00,59 ST,01/31/2021,12,7525089


In [74]:
turnstiles_daily[["PREV_DATE", "PREV_ENTRIES"]] = (turnstiles_daily
                                                       .groupby(["C/A", "UNIT", "SCP", "STATION","sample_num"])[["DATE", "ENTRIES"]]
                                                       .apply(lambda grp: grp.shift(1)))

In [75]:
def get_daily_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0? 
        print(row["ENTRIES"], row["PREV_ENTRIES"])
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter

# If counter is > 1Million, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits
# It'd probably be a good idea to use a number even significantly smaller than 1 million as the limit!
turnstiles_daily["DAILY_ENTRIES"] = turnstiles_daily.apply(get_daily_counts, axis=1, max_counter=1000000)

348 1237351900.0
658 1203534.0
3039842 777.0
43 3039842.0
127 1657032.0
2031759 368226.0
248 1264348.0
1294 6496220.0
327834 2642486.0
230 955727806.0
393225 7714275.0
57 2028804.0
172229024 8843948.0
157 3563454.0
262144 33554713.0
379 2018060.0
899 14003432.0
95 2407240.0
3834145 983905.0
160 3834145.0
1505257 366.0
29 1505299.0
1776 2313253.0
1245970758 1776.0
166 1246039062.0
0 67108864.0
54 9046801.0
458765 134277578.0
278 5239290.0
23157312 1627480738.0
369101672 1973089.0
589962 3166984.0
12 1098377.0
69 1021601.0
5 2706972.0
147 12840762.0
524337 2817588.0
697 13204039.0
61 3080422.0
16 6350317.0
262248 955056882.0
1055 4845991.0
57 1224795434.0
721366 6946581.0
458752 9259448.0
560 2384228.0
0 1955591.0
262296 8864207.0
909 3583402.0
569321681 16784233.0
772 7553900.0
5 13122450.0
21 2166896.0
139 655431311.0
654375162 10318322.0
67156129 10450185.0
1071773 4638441.0
277 2762132.0
1137 11047721.0
592403 4715175.0
32 6478782.0
285 3953135.0
13455725 855622861.0
855624180 134562

In [76]:
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,DATE,sample_num,ENTRIES,PREV_DATE,PREV_ENTRIES,DAILY_ENTRIES
0,A002,R051,02-00-00,59 ST,01/01/2021,11,7511647,NaN,NaN,NaN
1,A002,R051,02-00-00,59 ST,01/25/2020,0,7356730,NaN,NaN,NaN
2,A002,R051,02-00-00,59 ST,01/26/2020,0,7357340,01/25/2020,7356730.0,610.0
3,A002,R051,02-00-00,59 ST,01/27/2020,0,7358737,01/26/2020,7357340.0,1397.0
4,A002,R051,02-00-00,59 ST,01/28/2020,0,7360060,01/27/2020,7358737.0,1323.0


In [77]:
station_daily = turnstiles_daily.groupby(["STATION", "DATE","sample_num"])[['DAILY_ENTRIES']].sum().reset_index()
station_daily['DAY_OF_WEEK_NUM'] = pd.to_datetime(station_daily['DATE']).dt.dayofweek
station_daily.head(20)

,STATION,DATE,sample_num,DAILY_ENTRIES,DAY_OF_WEEK_NUM
0,1 AV,01/01/2021,11,0.0,4
1,1 AV,01/25/2020,0,0.0,5
2,1 AV,01/26/2020,0,5886.0,6
3,1 AV,01/27/2020,0,16326.0,0
4,1 AV,01/28/2020,0,17627.0,1
5,1 AV,01/29/2020,0,18010.0,2
6,1 AV,01/30/2020,0,18117.0,3
7,1 AV,01/30/2021,12,0.0,5
8,1 AV,01/31/2020,0,18682.0,4
9,1 AV,01/31/2021,12,3145.0,6


In [78]:
station_sample = turnstiles_daily.groupby(["STATION","sample_num"])[['DAILY_ENTRIES']].sum().reset_index()
#station_daily['DAY_OF_WEEK_NUM'] = pd.to_datetime(station_daily['DATE']).dt.dayofweek


In [82]:
station_sample.head(3)

,STATION,sample_num,DAILY_ENTRIES
0,1 AV,0,94648.0
1,1 AV,1,83166.0
2,1 AV,2,8470.0


In [83]:
station_sample.to_csv('station_sample.csv')